In [ ]:
import pyaudio,wave
import ollama
import warnings
import os
import glob
import sys
import torch
import re
import logging
os.environ['HF_ENDPOINT']='hf-mirror.com'#hugging_face镜像
# os.environ['HF_HUB_OFFLINE']='0'

#清理内存
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
if hasattr(torch.cuda, 'empty_cache'):
    torch.cuda.empty_cache()
del torch
warnings.filterwarnings("ignore")

In [ ]:
# import gradio
# sys.path.append('../Gradio_Python')
# from gradio import predict

# #文转语音
# import pyttsx3
# sys.path.append('../text_to_voice')
# from pyttsx3_def import text_voice##
# engine = pyttsx3.init()
# engine = text_voice(engine,rate=130)
# (r,ve,vs) = (engine.getProperty('rate'),engine.getProperty('volume'), engine.getProperty('voices'))
# print('目前语速是：',r,'\t','语音音量是:',ve)

#导入语音合成模型
sys.path.append('../Speech_Synthesis\GPT_Sovits\GPT-SoVITS-beta\GPT-SoVITS-beta0306fix\GPT_SoVITS')
from inference_maple import handle

In [ ]:
#声音转文本模型加载
model_choose='sherpa_onnx'#fast_whisper, whisper, sherpa_onnx
# model_choose='faster_whisper'
# model_choose='whisper'

if model_choose=='faster_whisper':
    sys.path.append('../voice_record')
    from voice_record_def import listen#记录输入的语音
    sys.path.append('../Speech_to_Text')
    from Fast_whisper.Fast_whisper import fast_whisper_opt
    from faster_whisper import WhisperModel

    model_size = "small"
    # model_size='large-v3'
    download_root=r'E:\LargeModel\Speech_to_Text\fast_whisper'
    #当download_root与模型在同一个目录时候
    model = WhisperModel(download_root+'\\'+model_size, device="cuda", compute_type="float16", download_root=download_root, local_files_only=False)

#whisper十分垃圾
elif model_choose=='whisper':
    # #whisper
    import whisper
    sys.path.append('../voice_record')
    from voice_record_def import listen#记录输入的语音
    sys.path.append('../Speech_to_Text')
    from Whisper.Whisper import whisper_output
    model_path = r'E:\LargeModel\Speech_to_Text\Whisper\Model'
    # model= whisper.load_model(name='small', download_root=model_path)#device='cuda:0'
    model= whisper.load_model(name='small', download_root=model_path)#device='cuda:0'

elif model_choose == 'sherpa_onnx':
    import sherpa_onnx
    path = 'E:/LargeModel/kaldi/sherpa_onnx_model/stream_zipformer'
    sys.path.append(path)
    import microphone_endpoint
    args_dict = {  
        'tokens': path+'/tokens.txt',  
        'encoder': path+'/encoder-epoch-99-avg-1.onnx',  
        'decoder': path+'/decoder-epoch-99-avg-1.onnx' ,  
        'joiner': path+'/joiner-epoch-99-avg-1.onnx',    
    } 
    #检查设备，顺便输出recongnizer
    check_recognizer= microphone_endpoint.main(args_dict)
else:
    print('Choose unfound asr model！')


#文本对话初始化
text_text_model='gemma:2b'

In [ ]:
#语音合成模型初始化
ref_wav_path="E:\LargeModel\Speech_Synthesis\GPT_Sovits\GPT-SoVITS-beta\派蒙--官方给的\参考音频\说话—既然罗莎莉亚说足迹上有元素力，用元素视野应该能很清楚地看到吧。.wav"
prompt_text= '既然罗莎莉亚说足迹上有元素力，用元素视野应该能很清楚地看到吧。'
prompt_language='zh'
text= '晚上好，祝你有一个甜美的睡眠！'
text_language='zh'
ref_free=False
stream=False
start=False
top_k=5
top_p=1
temperature=1
how_to_cut='按中英文句末标识符切'#输入按照句号切分，结合gemma文本每一段转换一次使用
sovits_path = r"E:\LargeModel\Speech_Synthesis\GPT_Sovits\GPT-SoVITS-beta\GPT-SoVITS-beta0306fix\GPT_SoVITS\pretrained_models\s2G488k.pth"
gpt_path = r"E:\LargeModel\Speech_Synthesis\GPT_Sovits\GPT-SoVITS-beta\GPT-SoVITS-beta0306fix\GPT_SoVITS\pretrained_models\s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt"

def tts(ref_wav_path, prompt_text, prompt_language, outword, text_language, sovits_path, gpt_path, start=start, top_k=top_k, top_p=top_p,temperature=temperature,how_to_cut=how_to_cut, ref_free = ref_free, stream=stream):
    # 将标准输出流重定向到空设备
    original_stdout =  sys.stdout
    sys.stdout = open('log_maple.txt', 'w')
    handle(ref_wav_path, prompt_text, prompt_language, outword, text_language, sovits_path, gpt_path, start=start, top_k=top_k, top_p=top_p,temperature=temperature,how_to_cut=how_to_cut, ref_free = ref_free, stream=stream)
    sys.stdout = original_stdout
    logging.getLogger().setLevel(logging.WARNING)


tts(ref_wav_path, prompt_text, prompt_language, text, text_language, sovits_path, gpt_path, start=start, top_k=top_k, top_p=top_p,temperature=temperature,how_to_cut=how_to_cut, ref_free = ref_free, stream=stream)

In [ ]:
#拜拜结束对话
#对话前删除临时录音文件
def delete_files(folder_path):
    file_list = glob.glob(folder_path + '/*')
    for file_path in file_list:
        if os.path.isfile(file_path):
            if '.' in file_path and file_path.split('.')[0].split('/')[-1][:4]=='test':
                os.remove(file_path)

# 调用函数删除文件夹中的文件
audio_file_path = 'test'
delete_files(audio_file_path)

#语音提醒可以对话了
begin_conversion ="您好，您现在可以跟我进行对话了！" 

tts(ref_wav_path, prompt_text, prompt_language, begin_conversion, text_language, sovits_path, gpt_path, start=start, top_k=top_k, top_p=top_p,temperature=temperature,how_to_cut=how_to_cut, ref_free = ref_free, stream=stream)

#用户说一次话并得到机器回复即为一轮。用户一轮说话中每有一个相对长的间隔，则将前面的录音分割为独立的一段，这一段视为一波录音。
outfile_name=audio_file_path+'\\test.mp3'
lc =0 #对话轮次，用于录音文件的名称存储
while(True):
    filenum=0 #每轮对话的录音波次
    tag=0 #一轮对话开始的标志
    sign=0 #whisper类是否转换声音了
    lc_result=''#每一轮次的语音输入结果

    while(model_choose !='sherpa_onnx'):
        temp=outfile_name.split('.')
        #当前波次有说话时，若后面说话间隔超过delayTime时进行下一波次录音；若当前波次还没有语音，当时间间隔超过tendure时候结束本轮录音。

        #增加用户的反应时间，3s太短了。
        sign, tag=listen(temp[0]+str(lc)+str('-')+str(filenum)+'.'+temp[1], tag, delayTime=0.8, tendure=5, mindb = 2000)
        # print(tag)
        if sign==1:#一轮对话录音结束的标志,每轮最后一波不转文字
            # if tag==0: #如果第一波录音算是反应时间
            #     continue
            break
        else:
            #将该波次语音转为文字
            # print(1,end='')
            open_audio_file = '/'.join(outfile_name.split('//'))
            open_audio_file=open_audio_file.split('.')
            open_audio_file = open_audio_file[0]+str(lc)+str('-')+str(filenum)+'.'+open_audio_file[1]
            # print(1,end='')
            #---------------有声音，但是不是话语时候，fast_whisper会输出随机的词语。
            if model_choose=='faster_whisper':
                result = fast_whisper_opt(model, open_audio_file, initial_prompt=None)
            #------------------whisper听到声音，但是这些声音是语气词无法翻译成话语时候会出错。
            elif model_choose=='whisper':
                result = whisper_output(model, open_audio_file, initial_prompt=None)
            #当对话是刚开始的时候
            # print(2,end='')
    
            lc_result += result
            # lc_result += ','
            if re.sub(r'[ .。！!?？-]','', result) in {'拜拜', '再见','byebye', 'Byebye' ,'掰掰'}:
                print(result)
                lc=-1
                break

            print(result, end='')
        filenum += 1

    #注意：每轮最后一波录音是空白

    if sign==1 and tag==0:#当用户开始说话但是无语音输入时候，退出整个对话
        print('\n', '*'*40)
        print(' '*5, '您的对话已超时，请重新对话！')
        print('\n', '*'*40)
        break
    #----------------------------------------------------------------------------#
    #shepra_onnx asr模型，快，但是不那么准确
    #输入声音但是无法翻译成话语时候会直接结束对话
    if model_choose == 'sherpa_onnx':
        # print('用户:  请输入你的问题')
        last_result, lc_result = microphone_endpoint.main(args_dict, False, check_recognizer)
        if last_result in {'拜拜'}:
            lc=-1
            result=last_result
        
        if len(lc_result)==0:
            print('\n', '*'*40)
            print(' '*5, '您的对话已超时，请重新对话！')
            print('\n', '*'*40)
            break

    print()
    #每轮的文本互动开始
    print('Bot:',end='')
    if lc==-1:
        print(result)
        break
    lc +=1

    #文本对话
    stream = ollama.chat(
        model=text_text_model,
        messages=[{'role': 'user', 'content': lc_result}],
        stream=True,
    )

    outword=""
    outword3=""

    max_chunk=4#每max_chunk轮播放一次
    chunk_num=1
    for chunk in stream:
        outword3 = chunk['message']['content']
        #打印对话内容
        print(outword3, end='', flush=True)
        outword += outword3
        #语音播放
        if outword3 in {'\n', ',','.','。','，','!','！', '?', '？',':', '：'}:
            #语音合成
            # 将标准输出流重定向到空设备
            #每max_chunk句就读一次
            #chunk_num==max_chunk:

            #每换一次行第一次。
            if outword3=='\n':
                tts(ref_wav_path, prompt_text, prompt_language, outword, text_language, sovits_path, gpt_path, start=start, top_k=top_k, top_p=top_p, temperature=temperature,how_to_cut=how_to_cut, ref_free = ref_free, stream=stream)
                # engine.say(outword)
                outword=""
                chunk_num=1
            #gemma是一个字一个chunk
            else:
                chunk_num +=1
        # engine.runAndWait()

    #把没凑够max_chunk的话读出来
    if len(outword)!=0:
        tts(ref_wav_path, prompt_text, prompt_language, outword, text_language, sovits_path, gpt_path, start=start, top_k=top_k, top_p=top_p,temperature=temperature,how_to_cut=how_to_cut, ref_free = ref_free, stream=stream)
    print()